# Import Libraries

In [28]:
import pandas as pd
import numpy as np

#Machine Learning
import sklearn
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn import neighbors
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

#Modelling
import matplotlib.pyplot as plt

# Import Data

In [2]:
data = pd.read_csv('final_data.csv')

C:\Users\lukes\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data.head()

,Unnamed: 0,key_0,index,speech,year,as_speaker,proper_name,gender,party,party_group,government,age,Dominant_Topic,Perc_Contribution,Topic_Keywords,speeches_count
0,0,0,2145981,"['complet', 'agre', 'friend', 'alreadi', 'ment...",2017,False,Damian Green,Male,Conservative,Conservative,True,61.0,3.0,0.4356,"â, univers, credit, work, peopl, pension, amen...",359
1,1,1,2145982,"['call', 'gerald', 'jone', 'fella']",2017,False,John Bercow,Male,Conservative,Conservative,True,54.0,2.0,0.8247,"hon, debat, minist, gentleman, point, way, giv...",2276
2,2,2,2145983,"['recent', 'ass', 'made', 'trend', 'level', 's...",2017,False,Peter Dowd,Male,Labour,Labour,False,59.0,3.0,0.8905,"â, univers, credit, work, peopl, pension, amen...",192
3,3,3,2145984,"['support', 'aspir', 'bos', 'number', 'self', ...",2017,False,Damian Green,Male,Conservative,Conservative,True,61.0,7.0,0.5681,"â, fund, year, tax, budget, care, need, peopl,...",359
4,4,4,2145985,"['citizen', 'advic', 'report', 'august', 'said...",2017,False,Peter Dowd,Male,Labour,Labour,False,59.0,3.0,0.6912,"â, univers, credit, work, peopl, pension, amen...",192


# Data Manipulation

In [4]:
model_data = data.pivot_table(index='proper_name', columns='Dominant_Topic', aggfunc='count',fill_value=0)

In [5]:
model_data.columns = model_data.columns.droplevel()

In [6]:
model_data = model_data.iloc[: , [0,1,2,3,4,5,6,7]]

In [7]:
model_data = pd.merge(model_data, data, how='left', on="proper_name")

In [8]:
model_data = model_data.drop(['Unnamed: 0','year','gender','speech',
                              'as_speaker','key_0','index','party',
                              'age','speeches_count','government',
                              'Dominant_Topic','Topic_Keywords',
                             ], axis=1)

In [9]:
model_data.loc[model_data['party_group'] =='Labour', 'party_group'] = 0
model_data.loc[model_data['party_group'] =='Conservative', 'party_group'] = 1
model_data.loc[model_data['party_group'] =='Liberal Democrat', 'party_group'] = 2
model_data.loc[model_data['party_group'] =='SNP', 'party_group'] = 3
model_data.loc[model_data['party_group'] =='Other', 'party_group'] = 4

In [10]:
model_data = model_data.drop_duplicates(subset="proper_name")

In [11]:
model_data = model_data.drop('proper_name',axis=1)

# Modelling

#### Creating Training and Testing Data

In [12]:
X = model_data.drop('party_group',axis=1) 
y = model_data['party_group']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

#### Decision Tree

In [14]:
dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)
y_pred = dtc.predict(X_test)

acc = dtc.score(X_test, y_test)*100
print(f"Decision Tree Test Accuracy {round(acc, 2)}%")

Decision Tree Test Accuracy 46.26%


#### SVM

In [15]:
svm = SVC()
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)

acc = svm.score(X_test,y_test)*100
print(f"SVM Algorithm Test Accuracy {round(acc, 2)}%")

SVM Algorithm Test Accuracy 49.66%


C:\Users\lukes\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


#### KNN

In [37]:
knn = KNeighborsClassifier(n_neighbors = 5)  # n_neighbors means k
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

acc = knn.score(X_test, y_test)*100
print(f"5 neighbors KNN Score: {round(acc,2)}%")

5 neighbors KNN Score: 54.42%


In [23]:
param_grid = {'n_neighbors': np.arange(1,15)} 
knn = neighbors.KNeighborsClassifier()
knn_cv = GridSearchCV(knn, param_grid, cv=5)
knn_cv.fit(X,y)
knn_cv.best_params_

C:\Users\lukes\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


{'n_neighbors': 13}

In [44]:
df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
display(df['Predicted'].value_counts())
display(df['Actual'].value_counts())

1    77
0    65
4     3
3     2
Name: Predicted, dtype: int64

1    71
0    60
3    12
2     3
4     1
Name: Actual, dtype: int64